## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Fayed Nassor and Mohamad Minoneshan

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [ ]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [ ]:
diabetes = load_diabetes()
df_X = diabetes.data
s_y = diabetes.target

## Q1.2 Define a function that creates a linear least squares regression model and a function to predict a continuous value given a linear regression model
The first function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

The second function should take in two parameters, `beta_hat` - the model generated from the `get_linear_regression` function, and `df_X` - that has the attribute values at which we want to predict a continuous value.  We assume that the format and ordering of `df_X` used to create the model and `df_X` used to generate predictions are consistent. 

In [ ]:
def get_linear_regression_model( df_X, s_y ):
    # Add column of 1s to X for intercept term
    X = np.hstack([np.ones((len(df_X), 1)), df_X])
    y = s_y
    # Calculate beta_hat using least squares formula
    beta_hat = np.linalg.lstsq(X, y, rcond=None)[0]
    return beta_hat

In [ ]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

In [ ]:
def predict_linear_regression_value(beta_hat, df_X):
    # Add column of 1s to X for intercept term
    df_X = np.hstack([np.ones((df_X.shape[0], 1)), df_X])
    # Predict y using beta_hat and df_X
    y_pred = np.matmul(df_X, beta_hat)
    return y_pred


In [ ]:
predicted_vals = predict_linear_regression_value( beta_hat, pd.DataFrame(np.random.random((3,4))))
predicted_vals

array([4.03669521, 4.39502589, 3.90884703])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [ ]:
def partition_data( df_X, s_y, k ):
    n = len(df_X)
    indices = np.arange(n)
    np.random.shuffle(indices)
    fold_size = n // k
    dict_k_df_X = {}
    dict_k_s_y = {}
    for i in range(k):
        start, end = i * fold_size, (i + 1) * fold_size
        if i == k - 1: # last fold may have extra rows
            end = n 
        fold_indices = indices[start:end]
        dict_k_df_X[i+1] = df_X[fold_indices]
        dict_k_s_y[i+1] = s_y[fold_indices]
    return dict_k_df_X, dict_k_s_y

In [ ]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [ ]:
# Check fold sizes
k = 5
dict_k_df_X, dict_k_s_y = partition_data(df_X, s_y, k)

for i in range(1, k+1):
    print(f"Fold {i} length of dataframe is {len(dict_k_df_X[i])} and length of series is {len(dict_k_s_y[i])}")

total_rows = sum([len(dict_k_df_X[i]) for i in range(1, k+1)])
print(f"The sum of the number of elements in each fold is {total_rows} and there are {len(df_X)} rows in the original df")

Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 88 and length of series is 88
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 88 and length of series is 88
Fold 5 length of dataframe is 90 and length of series is 90
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [ ]:
def get_mae( s_y, s_y_hat):
    # your code here
    n = len(s_y)
    error = np.abs(s_y - s_y_hat)
    mae = np.sum(error) / n
    return mae

In [ ]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

You must use your helper functions that you wrote above, `get_linear_regression_model`, `predict_linear_regression_value` and `get_mae`. 

In [ ]:
mae = np.array([])
for k in dict_k_df_X.keys():
    
    # your code here
    train_data = np.concatenate([v for i, v in dict_k_df_X.items() if i != k])
    train_targets = np.concatenate([v for i, v in dict_k_s_y.items() if i != k])

    # Fit a linear regression model to the training data
    beta_hat = get_linear_regression_model(train_data, train_targets)

    # Use the model to predict the target values for the test data
    s_y_hat = predict_linear_regression_value(beta_hat, dict_k_df_X[k])

    # Calculate the mean absolute error for the test set

    mae = np.append( mae, get_mae(dict_k_s_y[k],s_y_hat) )  

In [ ]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 41.96, the max MAE is 47.47, and the mean MAE is 44.39


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [ ]:
iris = load_iris()
df_X = iris.data
s_y = iris.target

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [ ]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [ ]:
for i in range(1, k+1):
    print(f"Fold {i} length of dataframe is {len(dict_k_df_X[i])} and length of series is {len(dict_k_s_y[i])}")

total_rows = sum([len(dict_k_df_X[i]) for i in range(1, k+1)])
print(f"The sum of the number of elements in each fold is {total_rows} and there are {len(df_X)} rows in the original df")


Fold 1 length of dataframe is 30 and length of series is 30
Fold 2 length of dataframe is 30 and length of series is 30
Fold 3 length of dataframe is 30 and length of series is 30
Fold 4 length of dataframe is 30 and length of series is 30
Fold 5 length of dataframe is 30 and length of series is 30
The sum of the number of elements in each fold is 150 and there are 150 rows in the original df


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [ ]:
def get_acc( s_1, s_2 ):
    # your code here
    num_correct = (s_1 == s_2).sum()
    num_total = len(s_1)
    acc = num_correct / num_total
    return acc

In [ ]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [ ]:

min_impurity_decreases = np.array([0.1,0.25,0.3,0.4])
best_min_impurity = None
best_accuracy = 0

for k in dict_k_df_X.keys():
    print(f"Fold {k}")
    X_test = dict_k_df_X[k]
    y_test = dict_k_s_y[k]
    X_train = np.concatenate([v for i, v in dict_k_df_X.items() if i != k])
    y_train = np.concatenate([v for i, v in dict_k_s_y.items() if i != k])
    
    best_min_impurity_fold = None
    best_accuracy_fold = 0
    
    for min_impurity_decrease in min_impurity_decreases:
        clf = tree.DecisionTreeClassifier(min_impurity_decrease=min_impurity_decrease, criterion="gini")
        accuracies = []
        for j in range(4):
            inner_fold_size = len(X_train) // 4
            X_inner_test = X_train[j*inner_fold_size:(j+1)*inner_fold_size]
            y_inner_test = y_train[j*inner_fold_size:(j+1)*inner_fold_size]
            X_inner_train = np.concatenate([X_train[:j*inner_fold_size], X_train[(j+1)*inner_fold_size:]])
            y_inner_train = np.concatenate([y_train[:j*inner_fold_size], y_train[(j+1)*inner_fold_size:]])
            clf.fit(X_inner_train, y_inner_train)
            y_pred = clf.predict(X_inner_test)
            accuracy = get_acc(y_inner_test, y_pred)
            accuracies.append(accuracy)
        mean_accuracy = np.mean(accuracies)
        print(f"Testing {min_impurity_decrease:.2f} min impurity decrease")
        print(f"    Average accuracy over 4 folds is {mean_accuracy:.2f}")
        if mean_accuracy > best_accuracy_fold:
            best_accuracy_fold = mean_accuracy
            best_min_impurity_fold = min_impurity_decrease
    
    print(f"\nBest min impurity decrease for fold {k} is {best_min_impurity_fold:.2f}\n")
    
    clf = tree.DecisionTreeClassifier(min_impurity_decrease=best_min_impurity_fold, criterion="gini")
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = get_acc(y_test, y_pred)
    print(f"Accuracy for fold {k} is {accuracy:.2f}\n\n")
    
    if best_accuracy_fold > best_accuracy:
        best_accuracy = best_accuracy_fold
        best_min_impurity = best_min_impurity_fold

print(f"\nBest min impurity decrease is {best_min_impurity:.2f}\n")

Fold 1
Testing 0.10 min impurity decrease
    Average accuracy over 4 folds is 0.97
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.88
Testing 0.30 min impurity decrease
    Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
    Average accuracy over 4 folds is 0.27

Best min impurity decrease for fold 1 is 0.10

Accuracy for fold 1 is 0.90


Fold 2
Testing 0.10 min impurity decrease
    Average accuracy over 4 folds is 0.93
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.84
Testing 0.30 min impurity decrease
    Average accuracy over 4 folds is 0.58
Testing 0.40 min impurity decrease
    Average accuracy over 4 folds is 0.25

Best min impurity decrease for fold 2 is 0.10

Accuracy for fold 2 is 0.97


Fold 3
Testing 0.10 min impurity decrease
    Average accuracy over 4 folds is 0.93
Testing 0.25 min impurity decrease
    Average accuracy over 4 folds is 0.69
Testing 0.30 min impurity decrease
    Average accura

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [ ]:
# initialize an empty list to store accuracies
outer_fold_accuracies = []

# loop over the folds
for k in dict_k_df_X.keys():
    
    # initialize an empty list to store fold accuracies
    fold_accuracies = []
    
    # loop over candidate min_impurity_decrease values
    for min_impurity_decrease in [0.1, 0.25, 0.3, 0.4]:
        
        # initialize an empty list to store inner fold accuracies
        inner_fold_accuracies = []
        
        # loop over the inner folds
        for j in range(1, 5):
            
            # concatenate the training data and targets from the other inner folds
            train_data = np.concatenate([v for i, v in dict_k_df_X.items() if i != k or i == j])
            train_targets = np.concatenate([v for i, v in dict_k_s_y.items() if i != k or i == j])

            # fit a decision tree classifier to the training data using the candidate hyperparameter
            dtc = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease=min_impurity_decrease)
            dtc.fit(train_data, train_targets)
            
            # evaluate the model on the left-out inner fold test set
            inner_fold_accuracy = get_acc(dict_k_s_y[j], dtc.predict(dict_k_df_X[j]))
            inner_fold_accuracies.append(inner_fold_accuracy)
        
        # calculate the mean inner fold accuracy for this candidate hyperparameter
        mean_inner_fold_accuracy = np.mean(inner_fold_accuracies)
        
        # store the mean inner fold accuracy for this candidate hyperparameter
        fold_accuracies.append(mean_inner_fold_accuracy)
    
    # choose the best hyperparameter based on mean inner fold accuracy
    best_min_impurity_decrease = [0.1, 0.25, 0.3, 0.4][np.argmax(fold_accuracies)]
    
    # fit a decision tree classifier to the outer fold training data using the best hyperparameter
    dtc = tree.DecisionTreeClassifier(criterion='gini', min_impurity_decrease=best_min_impurity_decrease)
    dtc.fit(np.concatenate([v for i, v in dict_k_df_X.items() if i != k]), np.concatenate([v for i, v in dict_k_s_y.items() if i != k]))
    
    # evaluate the model on the outer fold test set and store the accuracy
    outer_fold_accuracy = get_acc(dict_k_s_y[k], dtc.predict(dict_k_df_X[k]))
    outer_fold_accuracies.append(outer_fold_accuracy)

# print the min, max, and mean accuracy of the outer fold test sets
print('Min outer fold accuracy:', np.min(outer_fold_accuracies))
print('Max outer fold accuracy:', np.max(outer_fold_accuracies))
print('Mean outer fold accuracy:', np.mean(outer_fold_accuracies))


Min outer fold accuracy: 0.9
Max outer fold accuracy: 1.0
Mean outer fold accuracy: 0.9533333333333334
